In [21]:
%%capture
!pip install transformers
!pip install datasets
!pip install torchvision
!pip install wandb
!pip install ipynb
!pip install s3fs

In [2]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

nvidia_smi.nvmlShutdown()

Total memory: 11811160064
Free memory: 11719409664
Used memory: 91750400


In [1]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [14]:
# Remove Folders:
import shutil
shutil.rmtree('wandb')
#shutil.rmtree('gbert_baseline_res')
#shutil.rmtree('hyper')
#shutil.rmtree('tmp_trainer')

OSError: [Errno 39] Directory not empty: 'files'

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
torch.cuda.is_available()

from ipynb.fs.full.eval_metrics import *
import pandas as pd
import json
import s3fs
from functools import reduce
#https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f

hello()

# Opening JSON file
with open('../credentials.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
    key = json_object["key"]
    secret = json_object["secret_key"]
    bucket_name = json_object["bucket_name"]

s3 = s3fs.S3FileSystem(anon=False,key=key,secret=secret)

Hello World!


In [4]:
model_name = "bert-base-german-cased"

# for the tryouts, just use the first fold, otherwise all 5 folds
train_test_number = [1]
#train_test_number = [1,2,3,4,5]
ansatz = "gBERTbase_baseline"

train_file_name = "train.csv"
test_file_name = "test.csv"
#train_file_name = "train_plus_neutral_germeval.csv"
#test_file_name = "test_plus_neutral_germeval.csv"

In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from sklearn.model_selection import train_test_split

with s3.open(f"{bucket_name}/KFOLD1/train.csv",'r') as file:
    data = pd.read_csv(file)
with s3.open(f"{bucket_name}/KFOLD1/test.csv",'r') as file:
    test_data = pd.read_csv(file)
data = data[["Text", "majority_vote"]]
test_data = test_data[["Text", "majority_vote"]]
data.rename(columns={'Text': 'text', 'majority_vote': 'labels'}, inplace=True)
test_data.rename(columns={'Text': 'text', 'majority_vote': 'labels'}, inplace=True)

    # Define pretrained tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
#model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # ----- 1. Preprocess data -----#
    # Preprocess data
X = list(data["text"])
y = list(data["labels"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=11)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    # Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [6]:
%%capture
! pip install optuna

In [6]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average="macro")
    precision = precision_score(y_true=labels, y_pred=pred,average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred,average="macro")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [7]:
from transformers import EarlyStoppingCallback, TrainerCallback, TrainingArguments

class MemorySaverCallback(TrainerCallback):
    "A callback that deleted the folder in which checkpoints are saved, to save memory"
    def __init__(self, run_name):
        super(MemorySaverCallback, self).__init__()
        self.run_name = run_name

    def on_train_begin(self, args, state, control, **kwargs):
        print("Removing dirs...")
        if os.path.isdir(f'./{self.run_name}'):
            import shutil
            shutil.rmtree(f'./{self.run_name}')
        else:
            print("\n\nDirectory does not exists")

In [8]:
args = TrainingArguments(
    f"optuna",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=False,
    overwrite_output_dir=True,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
)

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2), MemorySaverCallback("hyper")]
)

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-german-cased/snapshots/702774c02b32a4f360d5fea60ab034d64bf0141c/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file pytorch_model.bin from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-german-cased/snapshots/702774c02b32a4f360d5fea60ab034d64bf0141c/pytorch_model.b

In [10]:
def my_hp_space_optuna(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-2, log=True),
        "warmup_steps":  trial.suggest_float("warmup_steps", 0., 0.9, step=0.3),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4,8,16])
    }

def my_objective(metrics):
    return metrics["eval_f1"]

In [11]:
sa = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=20,
    hp_space=my_hp_space_optuna, 
    compute_objective=my_objective
)

[I 2023-02-27 17:23:49,420] A new study created in memory with name: no-name-f49cac02-bc7d-4d7b-8d21-9233dd5324e4
Trial: {'learning_rate': 2.927094822781355e-06, 'warmup_steps': 0.6, 'weight_decay': 0.08973103190124292, 'per_device_train_batch_size': 16}
loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-german-cased/snapshots/702774c02b32a4f360d5fea60ab034d64bf0141c/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
 

Removing dirs...


Directory does not exists


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.145300,0.586993,0.666667,0.333333,0.500000,0.400000
2,0.602100,0.557764,0.682796,0.838798,0.524194,0.450063
3,0.608000,0.534303,0.693548,0.709351,0.552419,0.514271
4,0.908100,0.522237,0.704301,0.717544,0.572581,0.549637
5,0.586200,0.517316,0.709677,0.705422,0.588710,0.577628


***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to optuna/run-0/checkpoint-47
Configuration saved in optuna/run-0/checkpoint-47/config.json
Model weights saved in optuna/run-0/checkpoint-47/pytorch_model.bin
tokenizer config file saved in optuna/run-0/checkpoint-47/tokenizer_config.json
Special tokens file saved in optuna/run-0/checkpoint-47/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
Saving model checkpoint to optuna/run-0/checkpoint-94
Configuration saved in optuna/run-0/checkpoint-94/config.json
Model weights saved in optuna/run-0/checkpoint-94/pytorch_model.bin
tokenizer config file saved in optun

eval/accuracy,▁▄▅▇█
eval/f1,▁▃▆▇█
eval/loss,█▅▃▁▁
eval/precision,▁█▆▆▆
eval/recall,▁▃▅▇█
eval/runtime,▁▆▄▃█
eval/samples_per_second,█▃▅▆▁
eval/steps_per_second,█▃▅▆▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁


...

eval/accuracy,▁█
eval/f1,▁█
eval/loss,▁█
eval/precision,█▁
eval/recall,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Removing dirs...


Directory does not exists


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.074400,0.636787,0.666667,0.333333,0.500000,0.400000
2,0.888100,0.667597,0.666667,0.333333,0.500000,0.400000


***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to optuna/run-19/checkpoint-47
Configuration saved in optuna/run-19/checkpoint-47/config.json
Model weights saved in optuna/run-19/checkpoint-47/pytorch_model.bin
tokenizer config file saved in optuna/run-19/checkpoint-47/tokenizer_config.json
Special tokens file saved in optuna/run-19/checkpoint-47/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 186
  Batch size = 8
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [12]:
# bei 20 Trials, LR angepasst
sa
#BestRun(run_id='3', objective=0.7942684766214179, hyperparameters={'learning_rate': 1.9253554405748544e-05, 'warmup_steps': 0.8999999999999999, 'weight_decay': 0.04247303983914754, 'per_device_train_batch_size': 4})

BestRun(run_id='3', objective=0.7942684766214179, hyperparameters={'learning_rate': 1.9253554405748544e-05, 'warmup_steps': 0.8999999999999999, 'weight_decay': 0.04247303983914754, 'per_device_train_batch_size': 4})

In [13]:
# bei 10 Trials
sa
#BestRun(run_id='6', objective=0.7995689655172413, hyperparameters={'learning_rate': 2.56187627557514e-05, 'warmup_steps': 0.6, 'weight_decay': 0.028360233123464355, 'per_device_train_batch_size': 4})

BestRun(run_id='6', objective=0.7995689655172413, hyperparameters={'learning_rate': 2.56187627557514e-05, 'warmup_steps': 0.6, 'weight_decay': 0.028360233123464355, 'per_device_train_batch_size': 4})

In [16]:
%%capture
!pip install plotly

In [18]:
import plotly